In [1]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from skimage.transform import resize
import numpy as np
from tensorflow.python.platform import gfile


BATCH_SIZE = 50

tf.logging.set_verbosity(tf.logging.WARN)

def get_resized_image(filename, height, width):
    img = np.load(os.path.join('data','images',filename+'.npy'))
    return resize(img, (height, width), anti_aliasing=False, mode='constant')

if not os.path.exists(os.path.join('data','features')):
    os.makedirs(os.path.join('data','features'))

Inception v3 model is used to extract feature - values of last layer before last softmax layer are computed, as suggested by authors of the model.

Images are resized to conform with model input size.

Small batch size is selected due to memory restrictions.

In [9]:
with tf.Graph().as_default():
    module = hub.Module("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1")
    height, width = hub.get_expected_image_size(module)
    index = pd.read_csv(os.path.join('data', 'index.csv'))
    input_tensor = tf.placeholder(tf.float32, [None, height, width, 3])
    output_tensor = module(input_tensor)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        for i in range(0, index.shape[0], BATCH_SIZE):
            batch = []
            filenames = []
            for j in range(i, i + BATCH_SIZE):
                batch.append(get_resized_image(index['filename'].loc[j], height, width))
                filenames.append(index['filename'].loc[j])
            features = sess.run(output_tensor,{input_tensor:np.stack(batch, axis=0)})
            for j in range(BATCH_SIZE):
                cnn_code = features[j,:]
                np.save(os.path.join('data','features',filenames[j]+'.npy'),cnn_code)
            if i % 3000 == 0:
                print("Batch number: {}".format(str(i)))

Batch number: 0
Batch number: 3000
Batch number: 6000
Batch number: 9000
Batch number: 12000
Batch number: 15000
Batch number: 18000
Batch number: 21000
Batch number: 24000
Batch number: 27000
Batch number: 30000
Batch number: 33000
Batch number: 36000
Batch number: 39000
Batch number: 42000
Batch number: 45000
Batch number: 48000
Batch number: 51000
Batch number: 54000
Batch number: 57000
